In [33]:
import sqlite3, requests, time

In [34]:
def getPolaczenieBaza():
    baza = sqlite3.connect('logs_database.db')
    return baza

def getAdresyIpZTabeli(nazwaTabeli):
    baza = getPolaczenieBaza()
    kursor = baza.cursor()
    odpowiedzBazy = kursor.execute(f'SELECT ip FROM {nazwaTabeli}')
    listaZAdresami = []
    for ipObiekt in odpowiedzBazy:
        for ip in ipObiekt:
            listaZAdresami.append(ip)
    
    baza.close()
    return listaZAdresami

def getWysokoscISzerokoscPoIp(listaAdresow):
    def usunPierwszeStoAdresow(input_list):
        new_list = input_list[:100]  
        input_list[:100] = []        
        return new_list

    listaPrzetlumaczonychAdresow = []
    while listaAdresow:  
        print("---------")
        print("Długość orginalnej listy adresow: ", len(listaAdresow))
        print("Pierwszy element orginalnej listy: ", listaAdresow[0])

        kolejneStoAdresow = usunPierwszeStoAdresow(listaAdresow)
        odpowiedzAPI = requests.post("http://ip-api.com/batch", json=kolejneStoAdresow).json()

        for przetlumaczonyAdres in odpowiedzAPI:
            if przetlumaczonyAdres["status"] == "success":
                listaPrzetlumaczonychAdresow.append({"query": przetlumaczonyAdres["query"], "lat": przetlumaczonyAdres["lat"], "lon": przetlumaczonyAdres["lon"]})
            else:
                listaPrzetlumaczonychAdresow.append({"query": przetlumaczonyAdres["query"], "lat": "", "lon": ""})

        print("Długość listy odpowiedzi: ", len(listaPrzetlumaczonychAdresow))
        print("Ostatni element listy odpowiedzi: ", listaPrzetlumaczonychAdresow[len(listaPrzetlumaczonychAdresow)-1])

        if listaAdresow:  
            print("Czekam...")
            time.sleep(1)  
            
    return listaPrzetlumaczonychAdresow

def aktualizujTabeleZAdresami(noweAdresy):
    listaPrzetlumaczonychAdresow = getWysokoscISzerokoscPoIp(noweAdresy)
    baza = getPolaczenieBaza()
    kursor = baza.cursor()
    
    print("---------")

    for adresObiekt in listaPrzetlumaczonychAdresow:
        ip = f"'{adresObiekt['query']}'"
        wys = adresObiekt['lat']
        szer = adresObiekt['lon']
        # insertDoBazy = f"INSERT INTO translated_addresses(ip, lat, lon) VALUES({ip}, {wys}, {szer})" if wys and szer else f"INSERT INTO translated_addresses(ip) VALUES({ip})"
        if wys and szer:
            # print(f'INSERT INTO translated_addresses (ip, lat, lon) VALUES ({ip}, {wys}, {szer})')
            kursor.execute(f'INSERT INTO translated_addresses (ip, lat, lon) VALUES ({ip}, {wys}, {szer})')
        else:
            # print(f'INSERT INTO translated_addresses (ip) VALUES ({ip})')
            kursor.execute(f'INSERT INTO translated_addresses (ip) VALUES ({ip})')

    print("---------")
    getAdresyIpZTabeli('translated_addresses')

    baza.close()
    return

def sprawdzAtualnoscAdresow():
    listaAktualnychAdresow = list(set(getAdresyIpZTabeli('apache_access_logs')).union(set(getAdresyIpZTabeli('nginx_logs'))))
    listaZapisanychAdresow = getAdresyIpZTabeli('translated_addresses')
    noweAdresy = set(listaAktualnychAdresow) - set(listaZapisanychAdresow)
    if len(noweAdresy) == 0:
        return
    else:
        aktualizujTabeleZAdresami(list(noweAdresy))
        return

In [ ]:
sprawdzAtualnoscAdresow()

In [36]:
import sqlite3
import requests
import time
import logging

logging.basicConfig(level=logging.INFO)

def polacz_z_baza():
    return sqlite3.connect('logs_database.db')

def pobierz_adresy_z_tabel(tabele):
    zapytanie = " UNION ".join([f"SELECT ip FROM {tabela}" for tabela in tabele])
    with polacz_z_baza() as baza:
        kursor = baza.cursor()
        return [wiersz[0] for wiersz in kursor.execute(f"SELECT DISTINCT ip FROM ({zapytanie})").fetchall()]

def pobierz_przetlumaczone_adresy():
    with polacz_z_baza() as baza:
        kursor = baza.cursor()
        return [wiersz[0] for wiersz in kursor.execute("SELECT ip FROM translated_addresses").fetchall()]

def przetlumacz_adresy(adresy):
    wynik = []
    for i in range(0, len(adresy), 100):
        partia = adresy[i:i+100]
        try:
            odpowiedz_api = requests.post("http://ip-api.com/batch", json=partia, timeout=10).json()
        except requests.RequestException as e:
            logging.error(f"Błąd podczas wywołania API: {e}")
            continue
        for wpis in odpowiedz_api:
            if wpis["status"] == "success":
                wynik.append((wpis["query"], wpis["lat"], wpis["lon"]))
            else:
                wynik.append((wpis["query"], None, None))
        time.sleep(1)  # Pauza na potrzeby limitu API
    return wynik

def aktualizuj_przetlumaczone_adresy(nowe_adresy):
    przetlumaczone_adresy = przetlumacz_adresy(nowe_adresy)
    with polacz_z_baza() as baza:
        kursor = baza.cursor()
        kursor.executemany(
            "INSERT INTO translated_addresses (ip, lat, lon) VALUES (?, ?, ?)",
            przetlumaczone_adresy
        )
        baza.commit()

def synchronizuj_adresy():
    obecne_adresy = set(pobierz_adresy_z_tabel(['apache_access_logs', 'nginx_logs']))
    zapisane_adresy = set(pobierz_przetlumaczone_adresy())
    nowe_adresy = list(obecne_adresy - zapisane_adresy)
    if nowe_adresy:
        aktualizuj_przetlumaczone_adresy(nowe_adresy)

synchronizuj_adresy()